In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from scipy.special import expit
from sklearn.tree import DecisionTreeClassifier

In [3]:
heart_data = pd.read_csv("heart_cleveland.csv")

In [4]:
heart_data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,condition
0,69,1,0,160,234,1,2,131,0,0.1,1,1,0,0
1,69,0,0,140,239,0,0,151,0,1.8,0,2,0,0
2,66,0,0,150,226,0,0,114,0,2.6,2,0,0,0
3,65,1,0,138,282,1,2,174,0,1.4,1,1,0,1
4,64,1,0,110,211,0,2,144,1,1.8,1,0,0,0


In [6]:
# Функция для расчета энтропии в выборке
def calculate_entropy(y):
    # Получаем уникальные классы и их частоты в выборке
    unique_classes, class_counts = np.unique(y, return_counts=True)
    total_samples = ...

    # Рассчитываем вероятности для каждого класса, 
    # в class counts уже лежит то, что нужно - массив из встречаемости классов
    probabilities = class_counts / 

    # Рассчитываем энтропию по формуле Шеннона, для логарифма можем использовать np.log2
    entropy = -np.sum( ... * ...)
    return entropy

# Функция для поиска оптимального разделения (сплита) на основе энтропии
def find_best_split(X, y):
    # Получаем количество образцов (m) и количество признаков (n)
    m, n = X.shape

    # Инициализируем переменные для хранения лучшего сплита
    best_entropy = float('inf')
    best_split = None

    # Перебираем все признаки
    for feature_index in range(n):
        # Получаем уникальные значения признака
        unique_values = np.unique(X[:, feature_index])

        # Перебираем все уникальные значения в качестве порога (threshold)
        for threshold in unique_values:
            # Создаем маску для разделения данных на две части
            mask_left = X[:, feature_index] <= threshold
            # маска - массив из True\False, который бы указывал к левой или правой части 
            # сплита относится элемент под этим индексом
            # логично, что все элементы, что не попали в левую группу, попадают в правую
            mask_right = ~mask_left

            # Рассчитываем энтропию для левой и правой частей
            left_group = y[mask_left]
            right_group = y[mask_right]

            entropy_left = calculate_entropy(...)
            entropy_right = calculate_entropy(...)

            # Рассчитываем среднюю взвешенную энтропию
            total_samples = ...
            # вес левой и правой группы = (сколько в группе элементов)/ (сколько элементов было до сплита)
            weight_left = ...
            weight_right = ...
            # считаем взвешенную энтропию сплита (нужно домножить энтропию каждой группы на ее вес)
            split_entropy = ...

            # Если текущий сплит улучшает энтропию, обновляем лучший сплит
            if split_entropy < best_entropy:
                best_entropy = split_entropy
                best_split = {
                    'feature_index': feature_index,
                    'threshold': threshold,
                    'entropy': split_entropy
                }

    return best_split # Возвращаем информацию о лучшем сплите

In [5]:
# Выделяем признаки и целевую переменную
X = heart_data.drop('condition', axis=1)
y = heart_data['condition']

In [19]:
class DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.tree = None

    def fit(self, X, y):
        self.tree = self._fit(X, y, depth=0)

    def _fit(self, X, y, depth):
        unique_classes = np.unique(y)

        # Критерий остановки 1: Если все элементы принадлежат одному классу
        if len(unique_classes) == 1:
            return {'class': unique_classes[0]}

        # Критерий остановки 2: Достигнута максимальная глубина
        if self.max_depth is not None and depth == self.max_depth:
            return {'class': self._most_common_class(y)}

        # Критерий остановки 3: Если нет признаков для разделения
        if X.shape[1] == 0:
            return {'class': self._most_common_class(y)}

        # Находим лучший сплит
        best_split = ...

        # Критерий остановки 4: Если не удалось найти оптимальный сплит
        if best_split is None:
            return {'class': self._most_common_class(y)}
        # кажется мы это видели в best_split?
        feature_index = ...
        threshold = ...

        # Разделяем данные на две части
        mask_left = X[:, feature_index] <= threshold
        mask_right = ~mask_left

        # Рекурсивно строим поддеревья
        subtree_left = self._fit(X[mask_left], y[mask_left], depth + 1)
        subtree_right = self._fit(X[mask_right], y[mask_right], depth + 1)

        return {
            'feature_index': feature_index,
            'threshold': threshold,
            'subtree_left': subtree_left,
            'subtree_right': subtree_right
        }

    def _most_common_class(self, y):
        unique_classes, counts = np.unique(y, return_counts=True)
        return unique_classes[np.argmax(counts)]

    def predict(self, X):
        predictions = np.array([self._predict_single(x, self.tree) for x in X])
        return predictions

    def _predict_single(self, x, node):
        if 'class' in node:
            return node['class']

        if x[node['feature_index']] <= node['threshold']:
            return self._predict_single(x, node['subtree_left'])
        else:
            return self._predict_single(x, node['subtree_right'])


In [20]:
# Разбиваем данные на обучающий и тестовый набор
X_train, X_test, y_train, y_test = 

# Обучаем дерево
tree_classifier = DecisionTree(max_depth=3)
tree_classifier.fit(X_train.values, y_train.values)

In [22]:
# Предсказываем на тестовых данных
y_pred = tree_classifier.predict(X_test.values)

# Оцениваем производительность модели
accuracy = ...
precision = ...
recall = ...

Давайте теперь сравним качество нашего самописного дерева с реализацией из sklearn

In [23]:
# Создаем и обучаем модель DecisionTreeClassifier из scikit-learn
sklearn_tree = ...

...

# Предсказываем на тестовых данных
y_pred_sklearn = sklearn_tree.predict(X_test)

# Оцениваем производительность модели

accuracy_sklearn = ...
precision_sklearn = ...
recall_sklearn = ...

In [24]:
print("Custom Decision Tree:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("\nSklearn Decision Tree:")
print("Accuracy:", accuracy_sklearn)
print("Precision:", precision_sklearn)
print("Recall:", recall_sklearn)

Custom Decision Tree:
Accuracy: 0.8266666666666667
Precision: 0.7857142857142857
Recall: 0.7586206896551724

Sklearn Decision Tree:
Accuracy: 0.8533333333333334
Precision: 0.8214285714285714
Recall: 0.7931034482758621


<h1>ПРОДВИНУТАЯ ЧАСТЬ<h1>

In [ ]:
class GradientBoostingClassifier:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=None, regularization=0.1):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.regularization = regularization
        self.trees = []

    def fit(self, X, y):
        # Инициализация предсказаний
        predictions = np.zeros(len(y))

        for _ in range(self.n_estimators):
            # Вычисление градиента
            gradient = y - expit(predictions)

            # Обучение дерева на остатках
            tree = DecisionTree(max_depth=self.max_depth)
            ...

            # Применение регуляризации
            tree.tree = self.apply_regularization(tree.tree)

            # Обновление предсказаний с учетом нового дерева
            predictions += self.learning_rate * tree.predict(X)

            # Сохранение дерева
            self.trees.append(...)

    def apply_regularization(self, tree):
        # Рекурсивно применяем регуляризацию ко всем узлам дерева
        if 'value' in tree:
            tree['value'] = tree['value'] / (1 + self.regularization)
        if 'subtree_left' in tree:
            tree['subtree_left'] = self.apply_regularization(tree['subtree_left'])
        if 'subtree_right' in tree:
            tree['subtree_right'] = self.apply_regularization(tree['subtree_right'])
        return tree

    def predict_proba(self, X):
        # Суммируем предсказания всех деревьев
        predictions = sum(... for tree in self.trees)
        # Применение сигмоиды для получения вероятностей
        probabilities = expit(predictions)
        return ...

    def predict(self, X, threshold=0.5):
        # Преобразование вероятностей в бинарные метки классов
        return (self.predict_proba(X) >= ...).astype(int)

In [ ]:
# Инициализируем и обучаем градиентный бустинг
gradient_boosting = ...
...

In [ ]:
# Предсказываем на тестовых данных
y_pred_gb = ...

# Оцениваем производительность модели
...

print("Gradient Boosting:")
print("Accuracy:", accuracy_gb)
print("Precision:", precision_gb)
print("Recall:", recall_gb)

Gradient Boosting:
Accuracy: 0.7666666666666667
Precision: 0.71875
Recall: 0.8214285714285714
